In [18]:
import pandas as pd

In [19]:
df_esg = pd.read_csv('real_tabulation_scraped_data.csv', sep='\t')
df_roe = pd.read_csv('roe_results_tab.csv', sep='\t')
df_roe = df_roe.iloc[29:]
df_merge = pd.merge(df_esg, df_roe, on='name', how='inner')
df_merge['Year'] = df_merge["date_x"].str.extract(r'(\d{4})')
df_merge.drop(['date_x', 'date_y'], axis=1, inplace=True)
df_merge = df_merge.dropna(subset=['roe'])
df_merge

,name,grade,country,industry,employees,roe,Year
0,"1-800-FLOWERS.COM, Inc.",22.6,United States of America,Retailing,"4,000",-2.07%,2024
1,1&1 AG,27.7,Germany,Telecommunication Services,"3,280",4.34%,2024
2,"10X Genomics, Inc.",22.5,United States of America,Pharmaceuticals,"1,259",-24.87%,2024
3,11 Bit Studios SA,16.3,Poland,Software & Services,299,0.23%,2023
4,1st Source Corp.,35.0,United States of America,Banks,"1,170",12.01%,2024
...,...,...,...,...,...,...,...
7069,"The Hyakugo Bank, Ltd.",27.5,Japan,Banks,"2,792",3.43%,2024
7070,"The Hyakujushi Bank, Ltd.",29.0,Japan,Banks,"2,147",3.24%,2023
7073,The Italian Sea Group SpA,30.2,Italy,Machinery,710,30.74%,2023
7074,The J. M. Smucker Co.,27.2,United States of America,Food Products,"9,000",-1.18%,2023


In [20]:
df_merge['roe_num'] = df_merge['roe'].str.replace('%', '')
df_merge['roe_num'] = pd.to_numeric(df_merge['roe_num'], errors='coerce')
df_merge.to_csv('merged_data.csv', sep='\t', index=False)
df_merge = df_merge[df_merge['roe_num'] > -100]
df_merge = df_merge[df_merge['roe_num'] < 100] 